# Testing

> Standardized testing framework for daisyUI components in Jupyter notebooks

In [ ]:
#| default_exp core.testing

In [ ]:
#| export
from typing import List, Optional, Union, Callable, Any, Dict, Tuple
from dataclasses import dataclass, field
from enum import Enum
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
import asyncio
from contextlib import contextmanager
import time

# Import cjm-tailwind-utils for dynamic CSS generation
from cjm_tailwind_utils.all import TailwindBuilder

In [ ]:
#| export
# Import our own modules
from cjm_fasthtml_daisyui.core.types import SemanticColor, ColorUtility
from cjm_fasthtml_daisyui.core.resources import DaisyUIResources, ResourcePresets
from cjm_fasthtml_daisyui.core.config import DaisyUIConfig, DaisyUITheme, ThemeConfig
from cjm_fasthtml_daisyui.core.colors import ColorBuilder, apply_semantic_colors

## Component Testing Framework

The testing framework provides a standardized way to test and showcase daisyUI components in Jupyter notebooks with FastHTML.

In [ ]:
#| export
class DisplayMode(str, Enum):
    """Component display modes for testing"""
    INLINE = "inline"      # Components side by side
    STACKED = "stacked"    # Components stacked vertically
    GRID = "grid"          # Components in a grid
    SECTIONS = "sections"  # Components in labeled sections

In [ ]:
#| export
@dataclass
class ComponentExample:
    """Container for a component example with metadata"""
    component: FT
    title: str
    description: Optional[str] = None
    code: Optional[str] = None
    props: Optional[Dict[str, Any]] = None

In [ ]:
#| export
class ComponentTester:
    """
    Standardized testing framework for daisyUI components in notebooks
    
    Provides an interactive environment for testing and showcasing components
    with theme switching, responsive preview, and documentation.
    """
    
    def __init__(
        self,
        title: str = "Component Showcase",  # TODO: Add description
        pico: bool = False,  # TODO: Add description
        config: Optional[DaisyUIConfig] = None,  # TODO: Add description
        use_cdn: bool = True,  # TODO: Add description
        port: Optional[int] = 8000,  # TODO: Add description
        available_themes: Optional[List[Union[DaisyUITheme, str]]] = None
    ):
        """
        Initialize the component tester
        
        Args:
            title: Title for the showcase page
            pico: Whether to include Pico CSS (should be False for daisyUI)
            config: daisyUI configuration (only used with local resources)
            use_cdn: Whether to use CDN resources (vs local)
            port: Port for the test server (defaults to 8000)
            available_themes: List of themes to show in selector (defaults to all when using CDN)
        """
        self.title = title
        self.use_cdn = use_cdn
        self.port = port
        
        # Store headers for later use in rendering
        if use_cdn:
            # CDN includes all themes, so we ignore config for theme selection
            self.available_themes = available_themes or list(DaisyUITheme)
            self.hdrs = ResourcePresets.development()
        else:
            # Local resources can be configured
            self.config = config or DaisyUIConfig()
            self.available_themes = self._extract_themes_from_config(self.config)
            self.hdrs = ResourcePresets.production()
            
            # Add configuration CSS for local resources
            if config:
                self.hdrs.insert(0, DaisyUIResources.inline_css(config.to_css()))
        
        # Create FastHTML app
        self.app, self.rt = fast_app(pico=pico, hdrs=self.hdrs)
        self.server = None
        self.examples: List[ComponentExample] = []
        self._setup_routes()
    
    def _extract_themes_from_config(
        self,
        config: DaisyUIConfig  # TODO: Add description
    ) -> List[str]:  # TODO: Add return description
        """Extract theme names from configuration"""
        themes = []
        for theme in config.themes:
            if isinstance(theme, ThemeConfig):
                theme_name = theme.name.value if isinstance(theme.name, DaisyUITheme) else theme.name
                themes.append(theme_name)
            elif isinstance(theme, DaisyUITheme):
                themes.append(theme.value)
            else:
                themes.append(str(theme))
        return themes
    
    def _setup_routes(
        self
    ): # TODO: Add type hint
        """Set up default routes"""
        @self.rt('/')
        def index(
        ): # TODO: Add type hint
            "TODO: Add function description"
            return self._render_showcase()
        
        @self.rt('/theme/{theme}')
        def set_theme(
            theme: str  # TODO: Add description
        ): # TODO: Add type hint
            """Route to change theme dynamically"""
            return Script(f'document.documentElement.setAttribute("data-theme", "{theme}")')
    
    def add(
        self,
        component: Union[FT, Callable[[], FT]],
        title: str,  # TODO: Add description
        description: Optional[str] = None,  # TODO: Add description
        code: Optional[str] = None,  # TODO: Add description
        **props
    ) -> "ComponentTester":  # TODO: Add return description
        """
        Add a component example to the showcase
        
        Args:
            component: Component or function that returns a component
            title: Title for this example
            description: Optional description
            code: Optional code snippet to display
            **props: Properties passed to the component if it's a function
        """
        if callable(component):
            comp = component(**props)
        else:
            comp = component
            
        self.examples.append(ComponentExample(
            component=comp,
            title=title,
            description=description,
            code=code,
            props=props
        ))
        return self
    
    def showcase(
        self,
        *components: FT,
        mode: DisplayMode = DisplayMode.SECTIONS,
        titles: Optional[List[str]] = None
    ) -> "ComponentTester":  # TODO: Add return description
        """
        Quick method to add multiple components
        
        Args:
            *components: Components to showcase
            mode: How to display the components
            titles: Optional titles for each component
        """
        for i, comp in enumerate(components):
            title = titles[i] if titles and i < len(titles) else f"Example {i+1}"
            self.add(comp, title)
        
        self.display_mode = mode
        return self
    
    def _render_showcase(
        self
    ) -> FT:  # TODO: Add return description
        """Render the complete showcase page"""
        # Note: FastHTML app already includes headers, so we just return the body content
        return Div(
            # Navigation bar with theme switcher
            self._render_navbar(),
            
            # Main content area
            Main(
                Div(
                    H1(self.title, cls=TailwindBuilder().text(size="4xl", weight="bold").m(8, "b").build()),
                    
                    # Component examples
                    self._render_examples(),
                    
                    cls=TailwindBuilder().m("auto", "x").p(4, "x").p(8, "y").build()
                ),
                cls=(TailwindBuilder()
                     .add_class("container")
                     .min_h("screen")
                     .add_class(ColorUtility.BACKGROUND.with_color(SemanticColor.BASE_100))
                     .build())
            ),
            
            # Footer with color palette
            self._render_footer()
        )
    
    def _render_navbar(
        self
    ) -> FT:  # TODO: Add return description
        """Render navigation bar with theme switcher"""
        # Get theme names as strings
        theme_names = []
        for theme in self.available_themes:
            if isinstance(theme, DaisyUITheme):
                theme_names.append(theme.value)
            else:
                theme_names.append(str(theme))
        
        return Nav(
            Div(
                # Logo/Title
                A(self.title, cls=TailwindBuilder().add_class("btn btn-ghost").text(size="xl").build()),
                
                # Theme switcher
                Div(
                    Label("Theme:", cls="label"),
                    Select(
                        *[Option(t.title(), value=t, selected=(i==0)) for i, t in enumerate(theme_names)],
                        cls="select select-bordered select-sm",
                        onchange="document.documentElement.setAttribute('data-theme', this.value)"
                    ),
                    cls=TailwindBuilder().flex().items("center").gap(2).build()
                ),
                
                cls=(TailwindBuilder()
                     .add_class("navbar")
                     .add_class(ColorUtility.BACKGROUND.with_color(SemanticColor.BASE_200))
                     .p(4, "x")
                     .build())
            ),
            cls=TailwindBuilder().position("sticky").inset(0, "top").z(50).build()
        )
    
    def _render_examples(
        self
    ) -> FT:  # TODO: Add return description
        """Render all component examples"""
        if not self.examples:
            return Div(
                P("No components added yet. Use .add() or .showcase() to add components."),
                cls=TailwindBuilder().add_class("text-base-content/70").build()
            )
        
        sections = []
        for example in self.examples:
            section = self._render_example(example)
            sections.append(section)
        
        # Apply display mode
        container_cls = {
            DisplayMode.INLINE: TailwindBuilder().flex(wrap="wrap").gap(4).build(),
            DisplayMode.STACKED: TailwindBuilder().space(6, "y").build(),
            DisplayMode.GRID: (TailwindBuilder()
                              .grid(cols=1)
                              .add_class("md:grid-cols-2 lg:grid-cols-3")
                              .gap(6)
                              .build()),
            DisplayMode.SECTIONS: TailwindBuilder().space(8, "y").build()
        }.get(getattr(self, 'display_mode', DisplayMode.SECTIONS), TailwindBuilder().space(8, "y").build())
        
        return Div(*sections, cls=container_cls)
    
    def _render_example(
        self,
        example: ComponentExample  # TODO: Add description
    ) -> FT:  # TODO: Add return description
        """Render a single component example"""
        elements = []
        
        # Title
        elements.append(H3(example.title, cls=TailwindBuilder().text(size="xl", weight="semibold").m(2, "b").build()))
        
        # Description
        if example.description:
            elements.append(P(example.description, cls=(TailwindBuilder()
                                                        .add_class("text-base-content/70")
                                                        .m(4, "b")
                                                        .build())))
        
        # Component preview
        elements.append(
            Div(
                example.component,
                cls=(TailwindBuilder()
                     .p(6)
                     .add_class(ColorUtility.BACKGROUND.with_color(SemanticColor.BASE_200))
                     .rounded("lg")
                     .build())
            )
        )
        
        # Code snippet
        if example.code:
            elements.append(
                Details(
                    Summary("View Code", cls=TailwindBuilder().cursor("pointer").text(weight="medium").m(4, "t").build()),
                    Pre(
                        Code(example.code, cls="language-python"),
                        cls=TailwindBuilder().add_class("mockup-code").m(2, "t").build()
                    ),
                    cls=TailwindBuilder().m(4, "t").build()
                )
            )
        
        return Section(*elements, cls="")
    
    def _render_footer(
        self
    ) -> FT:  # TODO: Add return description
        """Render footer with color palette"""
        return Footer(
            Div(
                H4("Color Palette", cls=TailwindBuilder().text(size="lg", weight="semibold").m(4, "b").build()),
                self._render_color_palette(),
                cls=(TailwindBuilder()
                     .add_class("footer footer-center")
                     .p(8)
                     .add_class(apply_semantic_colors(
                         bg=SemanticColor.BASE_200,
                         text=SemanticColor.BASE_CONTENT
                     ))
                     .build())
            )
        )
    
    def _render_color_palette(
        self
    ) -> FT:  # TODO: Add return description
        """Render the current theme's color palette"""
        colors = [
            ("primary", "Primary"),
            ("secondary", "Secondary"),
            ("accent", "Accent"),
            ("neutral", "Neutral"),
            ("base-100", "Base"),
            ("info", "Info"),
            ("success", "Success"),
            ("warning", "Warning"),
            ("error", "Error")
        ]
        
        swatches = []
        for color_class, label in colors:
            swatches.append(
                Div(
                    Div(cls=TailwindBuilder().w(12).h(12).rounded("lg").add_class(f"bg-{color_class}").build()),
                    Span(label, cls=TailwindBuilder().text(size="xs").m(1, "t").build()),
                    cls=TailwindBuilder().flex(direction="col").items("center").build()
                )
            )
        
        return Div(*swatches, cls=TailwindBuilder().flex(wrap="wrap").gap(4).justify("center").build())
    
    def start(
        self,
        open_browser: bool = False  # TODO: Add description
    ) -> "ComponentTester":  # TODO: Add return description
        """Start the test server"""
        if not self.server:
            self.server = JupyUvi(self.app, port=self.port)
            if open_browser:
                HTMX()  # This will display the link in Jupyter
        return self
    
    def stop(
        self
    ): # TODO: Add type hint
        """Stop the test server"""
        if self.server:
            self.server.stop()
            self.server = None
    
    def __enter__(self):
        """Context manager support"""
        return self.start()
    
    def __exit__(
        self,
        exc_type,  # TODO: Add type hint and description
        exc_val,  # TODO: Add type hint and description
        exc_tb  # TODO: Add type hint and description
    ):
        """Context manager cleanup"""
        self.stop()

## Quick Testing Utilities

Helper functions for rapid component testing:

In [ ]:
#| export
def quick_test(
    *components: FT,
    title: str = "Quick Test"  # Title for the test page
) -> ComponentTester:  # Started ComponentTester instance
    "Quick function to test components"
    tester = ComponentTester(title=title)
    tester.showcase(*components)
    return tester.start()

In [ ]:
#| export
def test_variants(
    component_fn: Callable,  # Function that creates the component
    variants: Dict[str, List[Any]],
    base_props: Optional[Dict[str, Any]] = None,
    title: str = "Variant Testing"  # Title for the test page
) -> ComponentTester:  # Started ComponentTester instance
    "Test multiple variants of a component"
    tester = ComponentTester(title=title)
    base_props = base_props or {}
    
    # Generate all combinations
    import itertools
    
    if variants:
        keys = list(variants.keys())
        values = list(variants.values())
        
        for combo in itertools.product(*values):
            props = {**base_props}
            variant_desc = []
            
            for i, key in enumerate(keys):
                props[key] = combo[i]
                variant_desc.append(f"{key}={combo[i]}")
            
            component = component_fn(**props)
            title = " + ".join(variant_desc)
            
            tester.add(component, title, props=props)
    else:
        # No variants, just test base
        component = component_fn(**base_props)
        tester.add(component, "Base", props=base_props)
    
    return tester.start()

In [ ]:
#| export
@contextmanager
def theme_test(
    themes: Optional[List[Union[DaisyUITheme, str]]] = None  # List of themes to test with (uses all themes if None)
):
    "Context manager for testing with specific themes"
    # When using CDN, we just control which themes are shown in the selector
    tester = ComponentTester(
        title="Theme Testing",
        available_themes=themes or list(DaisyUITheme)
    )
    
    try:
        yield tester
        tester.start()
    finally:
        # Keep server running in notebook context
        pass

## Interactive Component Builder

Helper for building components interactively in notebooks:

In [ ]:
#| export
class ComponentBuilder:
    """Interactive component builder for notebooks"""
    
    def __init__(
        self,
        title: str = "Component Builder"  # TODO: Add description
    ):
        "TODO: Add function description"
        self.tester = ComponentTester(title=title)
        self.current_component = None
        self.component_history = []
    
    def create(
        self,
        tag: str = "div",  # TODO: Add description
        *children,
        **attrs
    ) -> "ComponentBuilder":  # TODO: Add return description
        """Create a new component"""
        self.current_component = NotStr(tag, *children, **attrs)
        self.component_history.append(self.current_component)
        return self
    
    def add_class(
        self,
        *classes: str
    ) -> "ComponentBuilder":  # TODO: Add return description
        """Add classes to the current component"""
        if self.current_component and hasattr(self.current_component, 'attrs'):
            current_cls = self.current_component.attrs.get('cls', '')
            new_cls = f"{current_cls} {' '.join(classes)}".strip()
            self.current_component.attrs['cls'] = new_cls
        return self
    
    def add_child(
        self,
        child: FT  # TODO: Add description
    ) -> "ComponentBuilder":  # TODO: Add return description
        """Add a child to the current component"""
        if self.current_component:
            self.current_component.children = list(self.current_component.children) + [child]
        return self
    
    def preview(
        self,
        title: Optional[str] = None  # TODO: Add description
    ) -> "ComponentBuilder":  # TODO: Add return description
        """Preview the current component"""
        if self.current_component:
            title = title or f"Component {len(self.component_history)}"
            self.tester.add(self.current_component, title)
        return self
    
    def show(
        self
    ) -> ComponentTester:  # TODO: Add return description
        """Show all previewed components"""
        return self.tester.start()
    
    def reset(
        self
    ) -> "ComponentBuilder":  # TODO: Add return description
        """Reset the builder"""
        self.current_component = None
        self.component_history = []
        self.tester = ComponentTester(self.tester.title)
        return self

## Test Data Generators

Helper functions to generate test data for components:

In [ ]:
#| export
class TestData:
    """Generate test data for components"""
    
    @staticmethod
    def lorem(
        words: int = 10  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Generate lorem ipsum text"""
        lorem_words = [
            "lorem", "ipsum", "dolor", "sit", "amet", "consectetur",
            "adipiscing", "elit", "sed", "do", "eiusmod", "tempor",
            "incididunt", "ut", "labore", "et", "dolore", "magna",
            "aliqua", "enim", "ad", "minim", "veniam", "quis"
        ]
        result = []
        for i in range(words):
            result.append(lorem_words[i % len(lorem_words)])
        return " ".join(result).capitalize() + "."
    
    @staticmethod
    def image(
        width: int = 300,  # TODO: Add description
        height: int = 200,  # TODO: Add description
        category: str = ""  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Generate placeholder image URL"""
        if category:
            return f"https://picsum.photos/{width}/{height}?{category}"
        return f"https://picsum.photos/{width}/{height}"
    
    @staticmethod
    def avatar(
        size: int = 100,  # TODO: Add description
        seed: Optional[str] = None  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Generate avatar URL"""
        if seed:
            return f"https://api.dicebear.com/7.x/avataaars/svg?seed={seed}&size={size}"
        return f"https://api.dicebear.com/7.x/avataaars/svg?size={size}"
    
    @staticmethod
    def items(
        count: int = 5,  # TODO: Add description
        prefix: str = "Item"  # TODO: Add description
    ) -> List[str]:  # TODO: Add return description
        """Generate list of items"""
        return [f"{prefix} {i+1}" for i in range(count)]
    
    @staticmethod
    def table_data(
        rows: int = 5,  # TODO: Add description
        cols: int = 3  # TODO: Add description
    ) -> List[List[str]]:  # TODO: Add return description
        """Generate table data"""
        headers = [f"Column {i+1}" for i in range(cols)]
        data = []
        for r in range(rows):
            row = [f"Cell {r+1}-{c+1}" for c in range(cols)]
            data.append(row)
        return [headers] + data

## Usage Examples

Examples of using the testing framework:

### Basic Component Testing

In [ ]:
# Create a basic tester
tester = ComponentTester("Button Examples")

# Add some button examples
tester.add(
    Button("Primary", cls=TailwindBuilder().add_class("btn btn-primary").build()),
    "Primary Button",
    description="A primary action button"
)

tester.add(
    Button("Secondary", cls=TailwindBuilder().add_class("btn btn-secondary").build()),
    "Secondary Button",
    description="A secondary action button",
    code='Button("Secondary", cls=TailwindBuilder().add_class("btn btn-secondary").build())'
)

# Start the server (commented out for docs)
tester.start()
HTMX()  # Display the link

In [ ]:
tester.stop()

### Quick Testing

In [ ]:
# Quick test multiple components
tester = quick_test(
    Button("Click me", cls=TailwindBuilder().add_class("btn btn-primary").build()),
    Card(
        H2("Card Title", cls=TailwindBuilder().add_class("card-title").build()),
        P("Card content goes here"),
        cls=TailwindBuilder().add_class("card bg-base-100").shadow("xl").build()
    ),
    title="Quick Component Test"
)
tester.start()
HTMX()

In [ ]:
tester.stop()

### Variant Testing

In [ ]:
# Test button variants
def create_button(text="Button", color="primary", size="md", outline=False):
    tb = TailwindBuilder()
    tb.add_class("btn")
    if color:
        tb.add_class(f"btn-{color}")
    if size != "md":
        tb.add_class(f"btn-{size}")
    if outline:
        tb.add_class("btn-outline")
    
    return Button(text, cls=tb.build())

# Test all combinations (commented out for docs)
tester = test_variants(
    create_button,
    variants={
        "color": ["primary", "secondary", "accent"],
        "size": ["sm", "md", "lg"],
        "outline": [False, True]
    },
    base_props={"text": "Click me"},
    title="Button Variants"
)
tester.start()
HTMX()

In [ ]:
tester.stop()

### Theme Testing

In [ ]:
# Example: Test with specific themes
with theme_test([DaisyUITheme.CUPCAKE, DaisyUITheme.DRACULA]) as tester:
    tester.showcase(
        Button("Themed Button", cls=TailwindBuilder().add_class("btn btn-primary").build()),
        Card(
            H2("Themed Card", cls=TailwindBuilder().add_class("card-title").build()),
            P("This card adapts to the theme"),
            Div(
                Button("Action", cls=TailwindBuilder().add_class("btn btn-sm").build()),
                cls=TailwindBuilder().add_class("card-actions").justify("end").build()
            ),
            cls=TailwindBuilder().add_class("card bg-base-100").shadow("xl").p(4).build()
        )
    )
HTMX()

In [ ]:
tester.stop()

### Using Test Data

In [ ]:
# Generate test data
lorem_text = TestData.lorem(20)
avatar_url = TestData.avatar(seed="john-doe")
items = TestData.items(5, prefix="Menu Item")

print(f"Lorem text: {lorem_text[:50]}...")
print(f"Avatar URL: {avatar_url}")
print(f"Items: {items}")

Lorem text: Lorem ipsum dolor sit amet consectetur adipiscing ...
Avatar URL: https://api.dicebear.com/7.x/avataaars/svg?seed=john-doe&size=100
Items: ['Menu Item 1', 'Menu Item 2', 'Menu Item 3', 'Menu Item 4', 'Menu Item 5']


## Best Practices

1. **Use Context Managers**: The `theme_test` context manager ensures proper cleanup
2. **Test Variants**: Use `test_variants` to systematically test all combinations
3. **Add Descriptions**: Provide clear descriptions for each component example
4. **Include Code**: Show the code that generates each component
5. **Test Responsiveness**: Resize the browser to test responsive behavior
6. **Check Themes**: Always test components with multiple themes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()